In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [3]:
#훈련 데이터셋 로드
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [4]:
#테스트 데이터셋 로드
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size=64

train_dataloader= DataLoader(training_data, batch_size=batch_size)
test_dataloader= DataLoader(test_data, batch_size=batch_size)


In [6]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y:{y.shape}, Type is '{y.dtype}'")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]), Type is 'torch.int64'


In [7]:
import torch

gpu_id = 1  # 0 또는 1 등으로 바꿔도 돼

device = (
    torch.device(f"cuda:{gpu_id}") if torch.cuda.is_available()
    else torch.device("mps") if torch.backends.mps.is_available()
    else torch.device("cpu")
)

print(f"Using {device} device")


Using cuda:1 device


## 모델 정의

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        self.flatten=nn.Flatten()
        
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device=device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델학습 정의

In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #오차 계산
        pred=model(X)
        loss=loss_func(pred, y)
        
        #Back-propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"los: {loss:>7f} [{current:>5d}/{size:5>5d}]")

In [11]:
def test(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_batches =len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred=model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg Loss: {test_loss:>8f} \n")

## 모델 학습시키기

In [12]:
epoch = 10

for t in range(epoch):
    print(f"Epoch: {t+1}\n---------------------------------------")
    train(train_dataloader, model, loss_func=loss_func, optimizer=optimizer)
    test(test_dataloader, model, loss_func)
print("Done!")

Epoch: 1
---------------------------------------
los: 2.298837 [   64/60000]
los: 2.286325 [ 6464/60000]
los: 2.272744 [12864/60000]
los: 2.276958 [19264/60000]
los: 2.253979 [25664/60000]
los: 2.228571 [32064/60000]
los: 2.242194 [38464/60000]
los: 2.201168 [44864/60000]
los: 2.201428 [51264/60000]
los: 2.183203 [57664/60000]
Test Error: 
 Accuracy: 40.8%, Avg Loss: 2.168992 

Epoch: 2
---------------------------------------
los: 2.175580 [   64/60000]
los: 2.162346 [ 6464/60000]
los: 2.112620 [12864/60000]
los: 2.138907 [19264/60000]
los: 2.086221 [25664/60000]
los: 2.027897 [32064/60000]
los: 2.066167 [38464/60000]
los: 1.978501 [44864/60000]
los: 1.988814 [51264/60000]
los: 1.932276 [57664/60000]
Test Error: 
 Accuracy: 55.5%, Avg Loss: 1.916893 

Epoch: 3
---------------------------------------
los: 1.944908 [   64/60000]
los: 1.912656 [ 6464/60000]
los: 1.797997 [12864/60000]
los: 1.853505 [19264/60000]
los: 1.745606 [25664/60000]
los: 1.684580 [32064/60000]
los: 1.720612 [38464/

batch=32, epoch=10일 경우 소요시간 1분 55초 / 407mb gram 소요

batch=64, epoch=10 일 경우 소요시간 1분 37초 / 409mb gram 소요

batch=128, epoch=10 일 경우 소요시간 1분 25초 / 409mb gram 소요

## 모델 저장하기

In [13]:
torch.save(model.state_dict(), "checkpoints/model.pth")
print("Model saved...")

Model saved...


## 모델 불러오기

In [14]:
model = NeuralNetwork().to(device=device)
model.load_state_dict(torch.load("checkpoints/model.pth"))

<All keys matched successfully>

## 모델 추론하기

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
index = 80
x, y = test_data[index][0], test_data[index][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"


In [16]:
"""
def clear_gpu_memory():
    import gc
    import torch

    globals_to_clear = ['model', 'x', 'y', 'pred']  # 전역 네임스페이스에 등록된 이름도 삭제
    for name in globals_to_clear:
        if name in globals():
            globals()[name] = None

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("🧹 GPU memory cleared!")

clear_gpu_memory()
"""

'\ndef clear_gpu_memory():\n    import gc\n    import torch\n\n    globals_to_clear = [\'model\', \'x\', \'y\', \'pred\']  # 전역 네임스페이스에 등록된 이름도 삭제\n    for name in globals_to_clear:\n        if name in globals():\n            globals()[name] = None\n\n    gc.collect()\n    torch.cuda.empty_cache()\n    torch.cuda.ipc_collect()\n    print("🧹 GPU memory cleared!")\n\nclear_gpu_memory()\n'